## Objective
### Build a real-time or file-based transcription system.

### Install Required Libraries

In [37]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q sounddevice scipy ffmpeg-python
!sudo apt-get update
!sudo apt-get install -y ffmpeg libportaudio2 libportaudiocpp0 portaudio19-dev
!pip install noisereduce librosa

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\ffmpeg-master-latest-win64-gpl-shared\bin"

In [39]:
import os
print(os.environ['PATH'])

C:\Users\user\anaconda3;C:\Users\user\anaconda3\Library\mingw-w64\bin;C:\Users\user\anaconda3\Library\usr\bin;C:\Users\user\anaconda3\Library\bin;C:\Users\user\anaconda3\Scripts;C:\Program Files\Python313\Scripts\;C:\Program Files\Python313\;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files\Git\cmd;C:\Program Files\MySQL\MySQL Shell 8.0\bin\;C:\Users\user\AppData\Local\Microsoft\WindowsApps;C:\Users\user\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\user\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Python 3.13;;C:\ffmpeg\ffmpeg-master-latest-win64-gpl-shared\bin;C:\ffmpeg\ffmpeg-master-latest-win64-gpl-shared\bin;C:\ffmpeg\ffmpeg-master-latest-win64-gpl-shared\bin


### Importing Required modules

In [40]:
# STEP 2: Imports and Whisper setup
from IPython.display import display, Audio
# from google.colab import files
import sounddevice as sd
import scipy.io.wavfile as wav
import soundfile as sf
import librosa
import noisereduce as nr
import tempfile
import whisper
import tempfile
import os

### Upload Audio File or Record from Mic

In [75]:
# Load Whisper model
model = whisper.load_model("base")
print("✅ Whisper model loaded")

# Record from mic
def record_audio(duration=5, sample_rate=16000):
    print(f"🎤 Recording for {duration} seconds...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()
    temp_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    wav.write(temp_wav.name, sample_rate, recording)
    print(f"✅ Recording saved to {temp_wav.name}")
    return temp_wav.name


# Record audio from microphone
def check_input_device():
    devices = sd.query_devices()
    input_devices = [i for i, d in enumerate(devices) if d['max_input_channels'] > 0]
    if not input_devices:
        raise RuntimeError("No input audio device found. Please connect a microphone or upload a file.")
    print(f"Input devices available: {input_devices}")
    return input_devices[0]  

device_index = check_input_device()

def record_audio(duration=5, sample_rate=16000):
    print(f"🎤 Recording for {duration} seconds...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16', device=device_index)
    sd.wait()
    temp_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    wav.write(temp_wav.name, sample_rate, recording)
    print(f"✅ Recording saved to {temp_wav.name}")
    return temp_wav.name


def upload_audio_file():
    print("📂 Please upload a .wav or .mp3 file...")
    uploaded = files.upload()
    filename = next(iter(uploaded))
    print(f"✅ File uploaded: {filename}")
    return filename

✅ Whisper model loaded
Input devices available: [0, 1, 4, 5, 9, 11, 13, 15, 18, 20]


In [42]:
def check_input_device():
    devices = sd.query_devices()
    input_devices = [i for i, d in enumerate(devices) if d['max_input_channels'] > 0]
    if not input_devices:
        raise RuntimeError("No input audio device found. Please connect a microphone or upload a file.")
    print(f"Input devices available: {input_devices}")
    return input_devices[0]  # pick first available device

device_index = check_input_device()

def record_audio(duration=5, sample_rate=16000):
    print(f"🎤 Recording for {duration} seconds...")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16', device=device_index)
    sd.wait()



Input devices available: [0, 1, 4, 5, 9, 11, 13, 15, 18, 20]


### Transcribe Function with Timestamps

In [77]:
# STEP 4: Transcribe with timestamps
def transcribe_audio(path):
    print("📝 Transcribing...")
    result = model.transcribe(path)
    print("✅ Transcription complete!\n")

    
    for segment in result['segments']:
        start = segment['start']
        end = segment['end']
        text = segment['text']
        print(f"[{start:.2f}s - {end:.2f}s]: {text}")

    if 'text' in result and isinstance(result['text'], str):
        return result['text']
    else:
        print("⚠️ Warning: Could not extract full transcribed text.")
        return "Transcription text not available."

### Choose Mode (Record or Upload)

In [79]:
mode = input("Choose mode: [1] Upload file, [2] Record from mic: ")

if mode == "1":
    file_path = upload_audio_file()
elif mode == "2":
    duration = int(input("Enter recording duration in seconds: "))
    file_path = record_audio(duration)
else:
    raise ValueError("Invalid option")

# Transcribe and play back
transcribed_text = transcribe_audio(file_path)
display(Audio(file_path))

Choose mode: [1] Upload file, [2] Record from mic:  2
Enter recording duration in seconds:  5


🎤 Recording for 5 seconds...
✅ Recording saved to C:\Users\user\AppData\Local\Temp\tmpw1bbnsbg.wav
📝 Transcribing...
✅ Transcription complete!

[0.00s - 2.48s]:  Hello good afternoon, how are you?
[2.48s - 4.48s]:  Still raining there?
